In [1]:
import sys
import os
import warnings
os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
!pip install pymorphy2
import pymorphy2
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_ru')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 215.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 6.5 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_ru.zip.


In [3]:
import pandas as pd
import numpy as np
import time
import pyarrow as pa
import pyarrow.parquet as pq
import scipy
!pip install implicit
import implicit
import bisect
import sklearn.metrics as m
!pip install catboost
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

%matplotlib inline
sns.set_style('darkgrid')

In [5]:
!pip install feather-format >> none
!pip install faiss-cpu --no-cache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 17.2 MB/s eta 0:00:00


In [ ]:
site_data = pd.read_csv('/data/site_data.csv')

In [7]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
title_tfidf = tfidf_vectorizer.fit_transform(site_data['title'].fillna(''))
title_tfidf

<144371x81916 sparse matrix of type '<class 'numpy.float64'>'
	with 774743 stored elements in Compressed Sparse Row format>

In [8]:
als = implicit.approximate_als.FaissAlternatingLeastSquares(factors = 50, iterations = 30, use_gpu = False, \
       calculate_training_loss = False, regularization = 0.1)

In [9]:
%%time
als.fit(title_tfidf)

  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: user 2min 8s, sys: 22.6 s, total: 2min 30s
Wall time: 39.7 s


In [10]:
u_factors = als.user_factors 
d_factors = als.item_factors
[len(u_factors), len(d_factors)]

[81916, 144371]

In [11]:
%%time
url_dict = {url: idurl for url, idurl in zip(site_data['url_host'], range(len(site_data['url_host'])))}
inv_url_map = {v: k for k, v in url_dict.items()}
title_emb = pd.DataFrame(d_factors)
title_emb['url_host'] = title_emb.index.map(inv_url_map)
title_emb.to_csv('title_emb.csv', index=False)

CPU times: user 7.46 s, sys: 412 ms, total: 7.87 s
Wall time: 7.89 s


In [12]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
description_tfidf = tfidf_vectorizer.fit_transform(site_data['description'].fillna(''))
description_tfidf

<144371x84888 sparse matrix of type '<class 'numpy.float64'>'
	with 1275639 stored elements in Compressed Sparse Row format>

In [13]:
als = implicit.approximate_als.FaissAlternatingLeastSquares(factors = 50, iterations = 30, use_gpu = False, \
       calculate_training_loss = False, regularization = 0.1)

In [14]:
%%time
als.fit(description_tfidf)

  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: user 2min 5s, sys: 13.6 s, total: 2min 19s
Wall time: 36.3 s


In [15]:
u_factors = als.user_factors 
d_factors = als.item_factors
[len(u_factors), len(d_factors)]

[84888, 144371]

In [16]:
%%time
description_emb = pd.DataFrame(d_factors)
description_emb['url_host'] = description_emb.index.map(inv_url_map)
description_emb.to_csv('description_emb.csv', index=False)

CPU times: user 6.09 s, sys: 249 ms, total: 6.34 s
Wall time: 6.35 s


In [17]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
keywords_tfidf = tfidf_vectorizer.fit_transform(site_data['keywords'].fillna(''))
keywords_tfidf

<144371x94113 sparse matrix of type '<class 'numpy.float64'>'
	with 733894 stored elements in Compressed Sparse Row format>

In [18]:
als = implicit.approximate_als.FaissAlternatingLeastSquares(factors = 50, iterations = 30, use_gpu = False, \
       calculate_training_loss = False, regularization = 0.1)

In [19]:
%%time
als.fit(keywords_tfidf)

  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: user 1min 33s, sys: 12.5 s, total: 1min 45s
Wall time: 27.8 s


In [20]:
u_factors = als.user_factors 
d_factors = als.item_factors
[len(u_factors), len(d_factors)]

[94113, 144371]

In [21]:
%%time
keywords_emb = pd.DataFrame(d_factors)
keywords_emb['url_host'] = keywords_emb.index.map(inv_url_map)
keywords_emb.to_csv('keywords_emb.csv', index=False)

CPU times: user 4.7 s, sys: 132 ms, total: 4.83 s
Wall time: 4.84 s


In [22]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
info_tfidf = tfidf_vectorizer.fit_transform(site_data['info'].fillna(''))
info_tfidf

<144371x160890 sparse matrix of type '<class 'numpy.float64'>'
	with 2083077 stored elements in Compressed Sparse Row format>

In [23]:
als = implicit.approximate_als.FaissAlternatingLeastSquares(factors = 50, iterations = 30, use_gpu = False, \
       calculate_training_loss = False, regularization = 0.1)

In [24]:
%%time
als.fit(info_tfidf)

  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: user 3min 13s, sys: 20.5 s, total: 3min 34s
Wall time: 55.9 s


In [25]:
u_factors = als.user_factors 
d_factors = als.item_factors
[len(u_factors), len(d_factors)]

[160890, 144371]

In [26]:
%%time
info_emb = pd.DataFrame(d_factors)
info_emb['url_host'] = info_emb.index.map(inv_url_map)
info_emb.to_csv('info_emb.csv', index=False)

CPU times: user 7.34 s, sys: 126 ms, total: 7.47 s
Wall time: 7.47 s


In [27]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
title_description_tfidf = tfidf_vectorizer.fit_transform(site_data['title_description'].fillna(''))
title_description_tfidf

<144371x118602 sparse matrix of type '<class 'numpy.float64'>'
	with 1673920 stored elements in Compressed Sparse Row format>

In [28]:
als = implicit.approximate_als.FaissAlternatingLeastSquares(factors = 50, iterations = 30, use_gpu = False, \
       calculate_training_loss = False, regularization = 0.1)

In [29]:
%%time
als.fit(title_description_tfidf)

  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: user 2min 47s, sys: 19.5 s, total: 3min 7s
Wall time: 48.5 s


In [30]:
u_factors = als.user_factors 
d_factors = als.item_factors
[len(u_factors), len(d_factors)]

[118602, 144371]

In [31]:
%%time
title_description_emb = pd.DataFrame(d_factors)
title_description_emb['url_host'] = title_description_emb.index.map(inv_url_map)
title_description_emb.to_csv('title_description_emb.csv', index=False)

CPU times: user 7.33 s, sys: 141 ms, total: 7.47 s
Wall time: 7.48 s


In [32]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
title_keywords_tfidf = tfidf_vectorizer.fit_transform(site_data['title_keywords'].fillna(''))
title_keywords_tfidf

<144371x136072 sparse matrix of type '<class 'numpy.float64'>'
	with 1309164 stored elements in Compressed Sparse Row format>

In [33]:
als = implicit.approximate_als.FaissAlternatingLeastSquares(factors = 50, iterations = 30, use_gpu = False, \
       calculate_training_loss = False, regularization = 0.1)

In [34]:
%%time
als.fit(title_keywords_tfidf)

  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: user 2min 40s, sys: 22.7 s, total: 3min 3s
Wall time: 48.1 s


In [35]:
u_factors = als.user_factors 
d_factors = als.item_factors
[len(u_factors), len(d_factors)]

[136072, 144371]

In [36]:
%%time
title_keywords_emb = pd.DataFrame(d_factors)
title_keywords_emb['url_host'] = title_keywords_emb.index.map(inv_url_map)
title_keywords_emb.to_csv('title_keywords_emb.csv', index=False)

CPU times: user 7.42 s, sys: 150 ms, total: 7.57 s
Wall time: 7.59 s


In [37]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
description_keywords_tfidf = tfidf_vectorizer.fit_transform(site_data['description_keywords'].fillna(''))
description_keywords_tfidf

<144371x133024 sparse matrix of type '<class 'numpy.float64'>'
	with 1726703 stored elements in Compressed Sparse Row format>

In [38]:
als = implicit.approximate_als.FaissAlternatingLeastSquares(factors = 50, iterations = 30, use_gpu = False, \
       calculate_training_loss = False, regularization = 0.1)

In [39]:
%%time
als.fit(description_keywords_tfidf)

  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: user 2min 43s, sys: 16.3 s, total: 2min 59s
Wall time: 47.2 s


In [40]:
u_factors = als.user_factors 
d_factors = als.item_factors
[len(u_factors), len(d_factors)]

[133024, 144371]

In [41]:
%%time
description_keywords_emb = pd.DataFrame(d_factors)
description_keywords_emb['url_host'] = description_keywords_emb.index.map(inv_url_map)
description_keywords_emb.to_csv('description_keywords_emb.csv', index=False)

CPU times: user 6.17 s, sys: 150 ms, total: 6.32 s
Wall time: 6.33 s
